# Hyperparameter tuning of DistillBERT with Classification Head

This notebook aims to improve on the limitation of the paper,
https://doi.org/10.48550/arXiv.1905.05583, where the authors did not conduct hyperparameter tuning for their neural networks. For our project, we attempt to choose values for TRAIN_BATCH_SIZE and LEARNING_RATE variables used in 'DistillBERT_finetuning_2_target.ipynb' notebook, since learning rate affects convergence while training batch size affects generalisation of model. We used a subset of the training data, and utilized Bayesian Optimisation.

In [1]:
!pip install bayesian-optimization

In [2]:
# Import libraries
from google.colab import drive
import numpy as np
import datetime
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader, SequentialSampler
from transformers import DistilBertModel, DistilBertTokenizer, AdamW
import re

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Import data and extract mini-set (500 rows) for hyperparametertuning. train_split was created with shuffling so no need shuffle again.
drive.mount('/content/drive')
train_split = pd.read_csv('/content/drive/MyDrive/train_split.csv')

Mounted at /content/drive


In [8]:
# Extracting subset of train data
mini_set = train_split[0:500]
val_set = train_split[500:551]

# Convert to binary classification
def good_bad(row):
  if row < 5:
    return 0
  else:
    return 1

mini_set['rating'] = mini_set['rating'].apply(good_bad)
val_set['rating'] = val_set['rating'].apply(good_bad)

<ipython-input-8-696a810ce133>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_set['rating'] = mini_set['rating'].apply(good_bad)
<ipython-input-8-696a810ce133>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set['rating'] = val_set['rating'].apply(good_bad)


In [22]:
from bayes_opt import BayesianOptimization

# Model and custom dataset classes similar to main notebook
# Define Custom Dataset
class CustomDataset(Dataset):
    ''' Custom dataset class defined to create '''

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.Content = dataframe.Content.to_numpy()
        self.targets = dataframe.rating.to_numpy()
        self.max_len = max_len

    # __len__ and __getitem__ methods to create map-style dataset to be interfaced by torch DataLoader method
    def __len__(self):
        return len(self.Content)

    def __getitem__(self, index):
        # Data preprocessing code to convert to lower-cased, remove trailing whitespace, html tags and urls
        Content = str(self.Content[index]).lower()
        Content = re.sub(r'<[^>]+>', '', Content)
        Content = re.sub(r'https://\S+|www\.\S+', '', Content)
        Content = re.sub(r'br\s', '', Content)
        Content = " ".join(Content.split())

        rating = self.targets[index]

        # Tokenisation of text
        inputs = self.tokenizer.encode_plus(
            Content,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            pad_to_max_length=True,
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(rating, dtype=torch.int)
        }

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    # Note: DistilBERT outputs a tuple where the first element at index 0
    # represents the hidden-state at the output of the model's last layer.
    # It is a tensor of shape (batch_size, sequence_length, hidden_size=768)
    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

# Training Parameters
MAX_LEN = 512
EPOCHS = 5
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create validation set (fixed for all experiments)
test_params = {'batch_size': 1,
                'shuffle': False,
                'sampler': SequentialSampler(val_set),
                'num_workers': 0
                }
val_data = CustomDataset(val_set, tokenizer, MAX_LEN)
testing_loader = DataLoader(val_data, **test_params)


def train(lr_exponent_val, train_batch):
    # Ensure train_batch and lr_exponent_val are discrete
    train_batch = int(train_batch)
    lr_exponent_val = int(lr_exponent_val)
    lr = 1*10**-(lr_exponent_val)

    # Create Dataset and Dataloader
    paramtune_set = CustomDataset(mini_set, tokenizer, MAX_LEN)
    train_params = {'batch_size': train_batch,
                    'shuffle': True,
                    'num_workers': 0
                    }
    paramtune_loader = DataLoader(paramtune_set, **train_params)

    # Initialize model
    model = DistillBERTClass()
    model.to(device)

    # Creating the loss function and optimizer
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)

    # Training loop over mini_set
    for epoch in range(EPOCHS):
        model.train()
        for _,data in enumerate(paramtune_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Evaluating model accuracy over test set
    model.eval()
    n_correct,nb_val_examples = 0,0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask)
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            nb_val_examples+=targets.size(0)

    run_accu = (n_correct*100)/nb_val_examples

    return run_accu

# Parameters to tune (learning rate and train batch size)
pbounds = {
    'lr_exponent_val': (3, 7),
    'train_batch': (2,9)
    }

optimizer = BayesianOptimization(
    f=train,
    pbounds=pbounds,
    verbose=2,
    random_state=1,
)

# Run Bayesian Optimisation Algorithm
optimizer.maximize(init_points=15, n_iter=15)

|   iter    |  target   | lr_exp... | train_... |
-------------------------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


| 1         | 86.27     | 4.668     | 7.042     |
| 2         | 37.25     | 3.0       | 4.116     |
| 3         | 37.25     | 3.587     | 2.646     |
| 4         | 37.25     | 3.745     | 4.419     |
| 5         | 74.51     | 4.587     | 5.772     |
| 6         | 88.24     | 4.677     | 6.797     |
| 7         | 37.25     | 3.818     | 8.147     |
| 8         | 37.25     | 3.11      | 6.693     |
| 9         | 92.16     | 4.669     | 5.911     |
| 10        | 37.25     | 3.562     | 3.387     |
| 11        | 74.51     | 6.203     | 8.778     |
| 12        | 80.39     | 4.254     | 6.846     |
| 13        | 66.67     | 6.506     | 8.262     |
| 14        | 37.25     | 3.34      | 2.273     |
| 15        | 37.25     | 3.679     | 8.147     |


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


| 16        | 88.24     | 5.241     | 6.152     |


...

| 20        | 94.12     | 5.249     | 8.98      |


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


| 21        | 50.98     | 7.0       | 2.0       |


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


KeyboardInterrupt: 

From the results above, we can see that evaluation accuracy is above 90% for trials with learning rate 1e-6 (i.e. lr_exponent_val = 6). For this context, batch size seems to affect accuracy insignificantly.